In [4]:
import pandas as pd
import plotly_express as px


In [5]:
demographics = pd.read_csv("/Users/fernandosotres/Desktop/PYTHON/Demography/blank-app/world_bank_data.csv")

In [ ]:

# Rename columns: keep only the year if column matches the pattern 'YYYY [YRYYYY]'
demographics.columns = [
    col.split(' ')[0] if ' [YR' in col else col
    for col in demographics.columns]

In [13]:
# ...existing code...

# Identify year columns (those that are digits)
year_cols = [col for col in demographics.columns if col.isdigit()]

# Convert to float and round to 2 decimals
demographics[year_cols] = demographics[year_cols].apply(pd.to_numeric, errors='coerce').round(2)

# ...existing code...

In [49]:
print(demographics['Country Name'].unique())


['Africa Eastern and Southern' 'Arab World' 'Central African Republic'
 'East Asia & Pacific' 'European Union' 'High income'
 'Latin America & Caribbean' 'Low income' 'Lower middle income' 'Mexico'
 'Middle East & North Africa' 'Middle income' 'North America'
 'South Africa' 'South Asia' 'Spain' 'Sub-Saharan Africa'
 'Upper middle income' nan]


In [20]:
# Get all unique names in the 'Series Name' column
unique_series_names = demographics['Series Name'].unique().tolist()
# Line the names up in a single column
unique_series_names = pd.Series(unique_series_names)
# erase 9, 10 and 11 lines
unique_series_names = unique_series_names.drop([9, 10, 11]).reset_index(drop=True)
print(unique_series_names)

0                 Death rate, crude (per 1,000 people)
1              Life expectancy at birth, total (years)
2                                        Net migration
3             Fertility rate, total (births per woman)
4                                    Population, total
5             Rural population (% of total population)
6                                     Urban population
7    Total alcohol consumption per capita (liters o...
8                         Population growth (annual %)
dtype: object


In [63]:
# order the data frame by the column 'Country Name'
demographics = demographics.sort_values(by='Country Name')

In [52]:
import plotly.express as px

# Seleccionar columnas de años desde 1974 en adelante
year_cols = [col for col in demographics.columns if col.isdigit() and int(col) >= 1974]

# Lista de nombres a eliminar
to_remove = [
    'Upper middle income', 'Middle income', 'Lower middle income', 'Low income', 'High income'
]

# Filtrar para las dos series de interés y quitar los grupos de ingreso
df_fertility = demographics[
    (demographics['Series Name'] == 'Fertility rate, total (births per woman)') &
    (~demographics['Country Name'].isin(to_remove))
]
df_urban = demographics[
    (demographics['Series Name'] == 'Urban population') &
    (~demographics['Country Name'].isin(to_remove))
]

# Melt para formato largo
fertility_melted = df_fertility.melt(
    id_vars=['Country Name'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Fertility rate, total (births per woman)'
).dropna()

urban_melted = df_urban.melt(
    id_vars=['Country Name'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Urban population'
).dropna()

# Unir ambos dataframes por país y año
merged = pd.merge(
    fertility_melted,
    urban_melted,
    on=['Country Name', 'Year'],
    how='inner'
)

# Graficar dispersión diferenciando por país
# ...existing code...

fig = px.scatter(
    merged,
    x='Fertility rate, total (births per woman)',
    y='Urban population',
    color='Country Name',
    animation_frame='Year',
    title='Índice De Fertilidad vs  Población Urbanizada',
    labels={
        'Fertility rate, total (births per woman)': 'Tasa De Fertilidad (Hijos Por Mujer)',
        'Urban population': 'Población Urbanizada'
    }
)

# Fijar el rango del eje x y y para ver todos los puntos
fig.update_xaxes(range=[0, 7])
fig.update_yaxes(range=[0, merged['Urban population'].max() * 1.05])

# Hacer los puntos más grandes
fig.update_traces(marker=dict(size=12))

# Cambiar la leyenda de "Year=" a "Año="
for frame in fig.frames:
    frame.name = frame.name.replace("Year=", "Año=")
fig.layout.sliders[0].currentvalue.prefix = "Año="

fig.show()

In [53]:
import plotly.express as px

# Seleccionar columnas de años desde 1974 en adelante
year_cols = [col for col in demographics.columns if col.isdigit() and int(col) >= 1974]

# Lista de income groups a mostrar
income_groups = [
    'Upper middle income', 'Middle income', 'Lower middle income', 'Low income', 'High income'
]

# Filtrar para las dos series de interés y solo los grupos de ingreso
df_fertility_income = demographics[
    (demographics['Series Name'] == 'Fertility rate, total (births per woman)') &
    (demographics['Country Name'].isin(income_groups))
]
df_urban_income = demographics[
    (demographics['Series Name'] == 'Urban population') &
    (demographics['Country Name'].isin(income_groups))
]

# Melt para formato largo
fertility_melted_income = df_fertility_income.melt(
    id_vars=['Country Name'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Tasa De Fertilidad (Hijos Por Mujer)'
).dropna()

urban_melted_income = df_urban_income.melt(
    id_vars=['Country Name'],
    value_vars=year_cols,
    var_name='Year',
    value_name='Población Urbanizada'
).dropna()

# Unir ambos dataframes por grupo de ingreso y año
merged_income = pd.merge(
    fertility_melted_income,
    urban_melted_income,
    on=['Country Name', 'Year'],
    how='inner'
)

# Graficar dispersión diferenciando por grupo de ingreso
fig = px.scatter(
    merged_income,
    x='Tasa De Fertilidad (Hijos Por Mujer)',
    y='Población Urbanizada',
    color='Country Name',
    animation_frame='Year',
    title='Índice De Fertilidad vs  Población Urbanizada (Grupos de Ingreso)',
    labels={
        'Tasa De Fertilidad (Hijos Por Mujer)': 'Tasa De Fertilidad (Hijos Por Mujer)',
        'Población Urbanizada': 'Población Urbanizada',
        'Country Name': 'Grupo de Ingreso'
    }
)

fig.update_xaxes(range=[0, 7])
fig.update_yaxes(range=[0, merged_income['Población Urbanizada'].max() * 1.05])
fig.update_traces(marker=dict(size=12))

# Cambiar la leyenda de "Year=" a "Año="
for frame in fig.frames:
    frame.name = frame.name.replace("Year=", "Año=")
fig.layout.sliders[0].currentvalue.prefix = "Año="

fig.show()

In [60]:
import plotly.express as px

# Seleccionar columnas de años desde 1974 en adelante
year_cols = [col for col in demographics.columns if col.isdigit() and int(col) >= 1974]

# Lista de income groups a eliminar
income_groups = [
    'Upper middle income', 'Middle income', 'Lower middle income', 'Low income', 'High income'
]

# Filtrar solo la serie de fertilidad y quitar income groups
fertility = demographics[
    (demographics['Series Name'] == 'Fertility rate, total (births per woman)') &
    (~demographics['Country Name'].isin(income_groups))
]

# Melt para formato largo
fertility_long = fertility.melt(
    id_vars=['Country Name'],
    value_vars=year_cols,
    var_name='Año',
    value_name='Tasa de Fertilidad'
).dropna()

# Nueva lógica de clasificación
def clasificar_fertilidad(x):
    if x < 2.1:
        return "Bajo del límite de equilibrio"
    elif 2.1 <= x < 4:
        return "Sobre el límite del equilibrio"
    elif 4 <= x < 6.3:
        return "Duplicación"
    else:
        return "Triplicación"

fertility_long['Grupo Fertilidad'] = fertility_long['Tasa de Fertilidad'].apply(clasificar_fertilidad)

# Orden y colores personalizados
orden_grupos = [
    "Triplicación",
    "Duplicación",
    "Sobre el límite del equilibrio",
    "Bajo del límite de equilibrio"
]
colores = {
    "Triplicación": "green",
    "Duplicación": "blue",
    "Sobre el límite del equilibrio": "orange",
    "Bajo del límite de equilibrio": "red"
}

# Contar países por grupo y año
conteo = fertility_long.groupby(['Año', 'Grupo Fertilidad']).size().reset_index(name='Número de Países')
conteo['Grupo Fertilidad'] = pd.Categorical(conteo['Grupo Fertilidad'], categories=orden_grupos, ordered=True)

# Graficar barras animadas por año
fig = px.bar(
    conteo.sort_values('Grupo Fertilidad'),
    x='Grupo Fertilidad',
    y='Número de Países',
    color='Grupo Fertilidad',
    animation_frame='Año',
    category_orders={'Grupo Fertilidad': orden_grupos},
    color_discrete_map=colores,
    title='Distribución de países por grupo de tasa de fertilidad a través de los años',
    labels={
        'Grupo Fertilidad': 'Grupo de Tasa de Fertilidad',
        'Número de Países': 'Número de Países',
        'Año': 'Año'
    }
)

# ...existing code...

# Asegurarse de que los años estén ordenados de menor a mayor y sean tipo string para la animación
conteo['Año'] = conteo['Año'].astype(int)
conteo = conteo.sort_values('Año')
conteo['Año'] = conteo['Año'].astype(str)

# Graficar barras animadas por año (ahora en orden correcto)
fig = px.bar(
    conteo.sort_values(['Año', 'Grupo Fertilidad']),
    x='Grupo Fertilidad',
    y='Número de Países',
    color='Grupo Fertilidad',
    animation_frame='Año',
    category_orders={'Grupo Fertilidad': orden_grupos, 'Año': sorted(conteo['Año'].unique(), key=int)},
    color_discrete_map=colores,
    title='Distribución de países por grupo de tasa de fertilidad a través de los años',
    labels={
        'Grupo Fertilidad': 'Grupo de Tasa de Fertilidad',
        'Número de Países': 'Número de Países',
        'Año': 'Año'
    }
)
fig.show()